In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

In [2]:
import sklearn

In [3]:
import torch.utils.data as Data
    
BATCH_SIZE = 100

train_dataset = datasets.MNIST(root='./data/',train=True,transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
test_dataset = datasets.MNIST(root='./data/',train=False,transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))


train_loader = Data.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=True)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size=BATCH_SIZE,shuffle=False)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.infl_ratio=1
        self.layers=nn.Sequential(
            
            nn.Linear(784, 512*self.infl_ratio),
            nn.BatchNorm1d(512*self.infl_ratio),
            nn.Hardtanh(),
            
      #      nn.Linear(1024*self.infl_ratio,1024*self.infl_ratio),
      #      nn.BatchNorm1d(1024*self.infl_ratio),
      #      nn.Hardtanh(),
            
      #      nn.Linear(1024*self.infl_ratio,1024*self.infl_ratio),
       #     nn.BatchNorm1d(1024*self.infl_ratio),
      #      nn.Hardtanh(),
            
            nn.Linear(512*self.infl_ratio,10),
            nn.BatchNorm1d(10),
            nn.Hardtanh(),
            
            nn.Softmax(dim=0)
        )
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.layers(x)
        return x

model = Net()
model.cuda()

Net(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Hardtanh(min_val=-1.0, max_val=1.0)
    (3): Linear(in_features=512, out_features=10, bias=True)
    (4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Hardtanh(min_val=-1.0, max_val=1.0)
    (6): Softmax(dim=0)
  )
)

In [5]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train(epoch):
    model.train()
    prbar = tqdm(total=len(train_loader))
    prbar.set_description("training epoch"+str(epoch))
    for data, target in train_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        prbar.update(1)
        prbar.set_postfix(loss=loss.item())
    prbar.close()
   # print('Train Epoch: {} \tLoss: {:.4f}'.format(epoch, loss.item()))
    
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('Test :  Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
for epoch in range(1,5):
    train(epoch)
    test()

training epoch2:   1%|          | 6/600 [00:00<00:16, 35.57it/s, loss=2.27]

Test :  Accuracy: 9515/10000 (95.15%)



training epoch3:   1%|          | 4/600 [00:00<00:24, 23.87it/s, loss=2.27]

Test :  Accuracy: 9669/10000 (96.69%)



training epoch4:   1%|          | 6/600 [00:00<00:18, 31.75it/s, loss=2.27]

Test :  Accuracy: 9714/10000 (97.14%)



training epoch4: 100%|██████████| 600/600 [00:21<00:00, 27.92it/s, loss=2.27]


Test :  Accuracy: 9750/10000 (97.50%)



In [6]:
# Product Quantition
from sklearn.cluster import KMeans

k = 1
s = 10
for p in model.parameters():
    if p.dim() >=2:
        pp = p.clone()
        X = pp.detach().cpu().numpy()
        n= X.shape[1]
        dic = {}
        for i in range(s):
            start = (i)*(n//s)
            if i!= s-1:
                x= X[:,start:(i+1)*(n//s)]
            else:
                x= X[:,start:]
            
            kmeans = KMeans(n_clusters=k)
            kmeans.fit(x)
            y_kmeans = kmeans.predict(x)
            centers = kmeans.cluster_centers_
            newx = np.asarray([centers[i] for i in y_kmeans])
            if i!= s-1:
                p[:,start:(i+1)*(n//s)] = torch.from_numpy(newx)
            else:
                p[:,start:] = torch.from_numpy(newx)
            break
        break
test()        

Test :  Accuracy: 9699/10000 (96.99%)

